# Bart Webinar 4: Hands-on Exercise

In this exercise you will use BART with Python/NumPy to reconstruct prospectively under-sampled 2D k-space data. 

The general workflow of this notebook is as follows:- 
1. Import k-space data from hdf5 file. There are 2 separate arrays in the file: undersampled k-space data and a low-resolution reference scan with a fully sampled calibration region. 
1. Visualize the data for both the files in k-space and in image space
1. Perform coil-compression and estimate the sensitivity maps from the reference scan 
1. Use the sensitivity maps and the undersampled k-space data to reconstruct the image using parallel imaging

You will fill in the missing code snippets between the `TODO` comments to complete the workflow

The data were collected on a 3 Tesla Siemens Vida scanner at UT Austin with IRB approval and informed consent.

### Main references

Pruessmann KP, Weiger M, Scheidegger MB, Boesiger P. SENSE: sensitivity encoding for fast MRI. Magn Reson Med. 1999 Nov;42(5):952-62. PMID: 10542355.

Huang F, Vijayakumar S, Li Y, Hertel S, Duensing GR. A software channel compression technique for faster reconstruction with many channels. Magn Reson Imaging. 2008 Jan;26(1):133-41. doi: 10.1016/j.mri.2007.04.010. Epub 2007 Jun 15. PMID: 17573223.

Uecker M, Lai P, Murphy MJ, Virtue P, Elad M, Pauly JM, Vasanawala SS, Lustig M. ESPIRiT--an eigenvalue approach to autocalibrating parallel MRI: where SENSE meets GRAPPA. Magn Reson Med. 2014 Mar;71(3):990-1001. doi: 10.1002/mrm.24751. PMID: 23649942; PMCID: PMC4142121.

Iyer S, Ong F, Setsompop K, Doneva M, Lustig M. SURE-based automatic parameter selection for ESPIRiT calibration. Magn Reson Med. 2020 Dec;84(6):3423-3437. doi: 10.1002/mrm.28386. Epub 2020 Jul 19. PMID: 32686178.


**Authors**: [Sidharth Kumar](mailto:sidharth.kumar@utexas.edu), [Jon Tamir](mailto:jtamir@utexas.edu)  
**Institution**: UT Austin

### Prerequisites

Make sure the BART configuration is completed and these paths are set correctly.

`TOOLBOX_PATH=/Your/path/to/bart`

`PATH=$TOOLBOX_PATH:$PATH`

In addition to the more common scientific Python libraries, we will also use `h5py`. If you are missing it, uncomment the following cell

In [ ]:
# !pip install h5py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bart import bart
import h5py
%matplotlib inline

## Load the data
Load the data to be used for reconstruction, the data have 2 separate files for undersampled k-space data and the fully sampled (only in calibration region) reference data.

In [ ]:
with h5py.File('webinar4_exercise_data.h5', 'r') as F:
    ksp_und = np.array(F['ksp_und'])
    ksp_ref = np.array(F['ksp_ref'])
    
print('ksp_und shape:', ksp_und.shape)
print('ksp_ref shape:', ksp_ref.shape)

## Resize the reference data
Zero-pad the reference k-space data to the same shape as the undersampled k-space data so that the calculated sensitivity maps are of same dimension.

Use the `bart resize` tool and call the output `ksp_ref`.

In [ ]:
# zero-pad ksp_ref in the readout direction to match the shape of ksp_und
### TODO your code here


###

print('ksp_und shape:', ksp_und.shape)
print('ksp_ref shape:', ksp_ref.shape)

## Visualize the reference scan
Plot the k-space of the reference scan and observe that only the centeral region is sampled. Also notice that the frequency encode direction is fully sampled and in the phase encode we only have samples in the central region.

In [ ]:
# visualize the ref scan kspace
fig, axes = plt.subplots(2, 6, figsize=(16, 6))
plt.tight_layout() # This automatically puts space between plots and make it tidy
for i in range(2):
    for ax, index in zip(axes[i], range(12)):
        ax.imshow(np.abs(ksp_ref[...,i*6+index]),cmap='gray',vmax=.00002)
        ax.set_title('Coil #{}'.format(i*6 + index))

Now we will perform a root-sum-of-squares (RSS) reconstruction of reference scan.

Use the `bart fft` tool to do a 2D unitary inverse Fourier transform. Assign the output to `cimg_ref`.

In [ ]:
# ifft of the ref k-space data
### TODO your code here

###

print(cimg_ref.shape)

Plot the calibration images, and observe the low resolution in these images.

In [ ]:
# visualize the ref scan coil images
fig, axes = plt.subplots(2, 6, figsize=(16, 6))
plt.tight_layout() # This automatically puts space between plots and make it tidy
for i in range(2):
    for ax, index in zip(axes[i], range(12)):
        ax.imshow(np.abs(cimg_ref[...,i*6+index]),cmap='gray',vmax=.00004)
        ax.set_title('Coil #{}'.format(i*6 + index))

Now use the `bart rss` tool to perform RSS along the coil dimension. Call the output `rss_ref`.

Plot the result and observe the low-resolution in the phase-encode direction.

In [ ]:
### TODO your code here

###

# visualize the RSS image
fig = plt.figure(figsize=(8, 4))
ax = fig.add_axes([0,0,1,1])
ax.imshow(np.abs(rss_ref),cmap='gray')
ax.set_title('Reference scan RSS')

## PICS scale factor
The parallel imaging and compressed sensing, `bart pics` command typically scales the k-space data so that the RSS image has maximum value close to one. This is so that the regularization factor is more consistent across different input datasets. 

The scale factor is chosen based on the RSS reconstruction of a fully sampled auto-calibration region in the data:
```python
scale_factor = np.percentile(abs(rss_acs), 99)
```
where `np.percentile` selects the 99th percentile and `rss_acs` is the RSS reconstruction.

When the input data do not contain a fully sampled calibration region, the `pics` command will not be able to automatically compute the scale factor and it will default to 1.0. Therefore, it is useful to manually calculate the scale factor and pass it to the `pics` command through the `-w` option.

In [ ]:
# get the scale factor from the reference data
scale_factor = np.percentile(abs(rss_ref), 99)
print('Scale factor: {:.5f}'.format(scale_factor))

## Plot and observe the under-sampled k-space data
Now we will look at the under-sampled data.

In [ ]:
fig, axes = plt.subplots(2, 6, figsize=(16, 6))
plt.tight_layout() # This automatically puts space between plots and make it tidy
for i in range(2):
    for ax, index in zip(axes[i], range(12)):
        ax.imshow(np.abs(ksp_und[...,i*6+index]), cmap='gray', vmax=.00002)
        ax.set_title('Coil #{}'.format(i*6 + index))

## Calculate sampling pattern
In the above k-space plots, the undersampling is not clearly observed due to the size of the plot. We can confirm that the data are under-sampled by using the `bart pattern` command. This command calculates the sampling pattern by checking where the data are strictly non-zero.

Use the `bart pattern` command to calculate the sampling pattern. Call the output `mask`.  
_Hint:_ BART outputs are often complex-valued. Therefore you will need to cast the output to a real-value using `.real`. 

Calculate the accelaration factor as $R=\frac{N}{N_s}$, where $N$ is the k-space size and $N_s$ is the number of sampled points in the mask. Plot the mask to observe that indeed the kspace data is undersampled.

In [ ]:
# calculate the sampling pattern
### TODO your code here


###

fig, axes = plt.subplots(1, 1, figsize=(4, 4))
plt.tight_layout() # This automatically puts space between plots and make it tidy
axes.imshow(mask, cmap='gray')
plt.title('Acceleration Factor: R={:.2f}'.format(R))


## Visualize the under-sampled data in the image domain
Perform an RSS reconstruction of the under-sampled data, following similar steps to those of the reference scan. Use the expected variable names based on the code provided.

In [ ]:
# take IFFT of the under-sampled data
### TODO your code here

###

print(cimg_und.shape)

In [ ]:
# visualize the aliased images
fig, axes = plt.subplots(2, 6, figsize=(16, 6))
plt.tight_layout() # This automatically puts space between plots and make it tidy
for i in range(2):
    for ax, index in zip(axes[i], range(12)):
        ax.imshow(np.abs(cimg_und[...,i*6+index]),cmap='gray',vmax=.00004)
        ax.set_title('Coil #{}'.format(i*6 + index))

In [ ]:
# RSS zero-filled reconstruction
### TODO your code here

###

fig = plt.figure(figsize=(8, 4))
ax = fig.add_axes([0,0,1,1])
ax.imshow(np.abs(rss_und),cmap='gray')
ax.set_title('Under-sampled RSS')

## Coil Compression
Before reconstucting, we will coil-compress the coils using the Software Coil Compression method.

First calculate the coil compression matrix using `bart cc` command on the reference scan data. Call the coil compression matrix `cc_matrix`.

Then project the reference scan and the under-sampled k-space data using `bart ccapply` command to the desired number of virtual coils (we recommend about 8) for this data. Call the outputs based on what's shown in the cells.

To see more information about any of these commands use `!bart cc -h` in a new cell

In [ ]:
# run coil compression to get coil compression matrix
### TODO your code here

###

print(cc_matrix.shape)

In [ ]:
# apply coil compression to num_vcoils  virtual coils -- apply to ref and to kspace data!
### TODO your code here



###

print(ksp_cc_ref.shape)
print(ksp_cc_und.shape)

## Coil sensitvity map calibration
Use `bart ecalib` command to estimate the coil sensitivities from the coil compressed reference k-space data and plot the coil sensitivity maps. You can use the `-a` parameter to automatically estimate the ESPIRiT parameters. Call the output `coil_sens`.

In [ ]:
# bart ecalib help
!bart ecalib -h

In [ ]:
# estimate the coil sensitivities
### TODO your code here

###

print(coil_sens.shape)

In [ ]:
# visualize the estimated coil sensitivities
plt.figure(figsize=(20,6))
for index in range(num_vcoils):
    plt.subplot(1,num_vcoils,index+1)
    plt.imshow(np.abs(coil_sens[..., index]),cmap='gray')
print('coil sensitivity maps')

## Reconstruction
Reconstruct the coil-compressed data using the `bart pics` command. First, run the help command and observe which flags you will require for regularation options, manually scaling/re-scaling of the data, and input order. We recommend using $\ell_2$ regularization.

Plot the reconstructed image and observe that the aliasing is removed and you can see a high resolution T2-FLAIR image of brain.

In [ ]:
# bart pics command help
!bart pics -h

In [ ]:
# reconstruct the image using bart pics
### TODO your code here


###

print(recon.shape)

In [ ]:
# visualize the reconstructed T2-FLAIR image

fig = plt.figure(figsize=(8, 4))
ax = fig.add_axes([0,0,1,1])
ax.imshow(np.abs(recon), cmap='gray', vmax=.0003)
ax.set_title('T2 FLAIR reconstructed')

__Thank you for your participation!__

## (Extra credit) Undersampling by an additional factor of 2
In simulation, undersampling can be performed in both the directions i.e. phase and frequency encode. However on actual scanner, the frequency encode direction is always fully sampled. Therefore we will further undersample the phase encode direction by a factor of 2

In [ ]:
ksp_cc_und_4x = np.zeros((ksp_cc_und.shape),dtype=complex)

ksp_cc_und_4x[1::4,...] = ksp_cc_und[1::4,...] #undersampling in the phase encode direction
# ksp_cc_und_4x[:,1::2,:] = ksp_cc_und[:,1::2,:] #undersampling in the frequency encode direction

## Reconstruct
Follow a similar approach to the above exercise to reconstruct the 4x under-sampled data.